# Simulating a FHN network.

## Set parameters and define the network

In [1]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
using Random
N = 90
eps = 0.05
a = 0.5
b = -bmatrix(pi/2-0.1, eps)
σ = 0.0506
G = wattsstrogatzmatrix(N, 3, 1);#0.232)


LoadError: LoadError: InterruptException:
in expression starting at /Users/miguel/Documents/GitHub/Coupled-FHN/src/network_simulation.jl:1

In [408]:
# Set the initial conditions

x_0 = zeros(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3) #rand(Uniform(-a + a^3 / 3, a - a^3 / 3), N)

# Define the ODE problem
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 300.0), [a, eps, σ])

# Solve the ODE problem
# alg = Tsit5()
sol = solve(prob);


### Plots

In [410]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 600))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f


## Study Cluster Synchronization in the integration of the FHN model.

### First, a test

Based on what is shown in the paper (https://doi.org/10.48550/arXiv.2303.08668), first we test with a complete graph.

In [15]:
include("src/msf.jl")
zero_msf = msf_zero()
println("zero_msf = ", zero_msf)
println("msf = ", master_stability_function(zero_msf, 0))

zero_msf = 0.20373021436669891
msf = 

-1.8873508449642648e-17


In [100]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
include("src/cluster_synch.jl")
using Random
N = 10
eps = 0.05
a = 0.5
b = -bmatrix(pi/2-0.1, eps)
G = test_matrix_for_cluster_synch();
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
critical_couplings = unique(round.(zero_msf./eigenvalues, digits=8))[2:end]#unique(zero_msf./eigenvalues)[2:end]#
println("Critical Couplings: ", critical_couplings)
println("Eigenvalues: ", eigenvalues)
σ = critical_couplings[1]*0.9


Critical Couplings: [0.02037302, 0.01198413, 0.00970144]
Eigenvalues: [3.552713678800501e-15, 9.999999999999996, 9.999999999999996, 10.000000000000014, 16.99999999999999, 16.999999999999993, 17.000000000000007, 20.99999999999998, 20.999999999999993, 21.0]


0.018335718

In [101]:
x_0 = zeros(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)

prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])

sol = solve(prob; dtmax=0.5);


In [102]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

colors = ["#5ec962", "#21918c", "#3b528b", :red]
f = Figure(size= (1000, 600))
uni_clusters = unique(clusters)
for (i, cluster) in enumerate(uni_clusters)
    if isempty(cluster)
        continue
    else
        if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
            cluster = setdiff(cluster[1], uni_clusters[i+1][1])
        else
            cluster = cluster[1]
        end
        println("Cluster $i: $cluster")

        ax = Axis(f[i, 1])
        ax.xlabel = "Time"
        ax.ylabel = "Synch Error"
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $cluster", linewidth=1, color = colors[i])
        axislegend()
    end
end
display(f);

Cluster 1: [1, 2, 3]


Cluster 2: [4, 5, 6]
Cluster 3: [7, 8, 9, 10]


In [106]:
using GLMakie
using Trapz
using ProgressMeter
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

N_d = 2
N_realizations = 3
d_sweep = range(0.005, 0.02, length=N_d)
t_measurement = 500.0
t_transient = 500.0
synch_averages = zeros(N_d, 3)

f = Figure(size= (800, 600))
uni_clusters = unique(clusters)
@showprogress for (k, σ) in enumerate(d_sweep)
    for realization in 1:N_realizations
        x_0 = zeros(2*N) + 0.001 .* randn(2*N)
        prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, t_transient + t_measurement), [a, eps, σ])
        # alg = Tsit5()
        sol = solve(prob; dtmax=0.5);

        for (i, cluster) in enumerate(uni_clusters)
            if isempty(cluster)
                continue
            else
                if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
                    cluster = setdiff(cluster[1], uni_clusters[i+1][1])
                else
                    cluster = cluster[1]
                end
                t_values, synch_error = local_synch_error(sol, cluster)
                synch_averages[k, i] += trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
            end
        end
    end
end

ax = Axis(f[1, 1])
ax.xlabel = "Coupling Strength"
ax.ylabel = "Average Synch Error"
for i in 1:3
    cluster = uni_clusters[i]
    if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
        cluster = setdiff(cluster[1], uni_clusters[i+1][1])
    else
        cluster = cluster[1]
    end
    lines!(ax, d_sweep, synch_averages[:, i]; label="Cluster $cluster", linewidth=1)
end
vlines!(ax, critical_couplings; label="Critical Couplings", linewidth=1, color = :red)
axislegend()
display(f);


In [68]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 200))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

### Now for a watts strogatz network:

In [175]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 300))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

In [117]:
using Graphs

# Assuming you have an adjacency matrix as a 2D array
adjacency_matrix = [
    0 1 0 0;
    1 0 0 0;
    0 0 0 1;
    0 0 1 0;
]

# Convert the adjacency matrix to an undirected graph
graph = SimpleGraph(adjacency_matrix)

# Get connected components
components = connected_components(graph)

# Print the connected components
println("Connected Components:")
for (i, component) in enumerate(components)
    println("Component $i: ", component)
end



Connected Components:
Component 1: [1, 2]
Component 2: [3, 4]
